## SreeDananjay S(21BAI1807)
#### Speech and Language processing Lab assignment 10

In [1]:
!pip install librosa scikit-learn matplotlib numpy

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 1.8 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 3.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 3.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 MB 8.7 MB/s eta 0:00:00:00:0100:01m

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
#!/usr/bin/env/python
#
# A script to download bird sound files from the www.xeno-canto.org archives with metadata
# The program downloads all the files found with the search terms into
# subdirectory data/xeno-canto-dataset/searchTerm.
# and corresponding json files.

# Karoliina Oksanen, 2014
# Updated to python 3.7.4, Agnieszka Mikolajczyk, 2019
# Added json download, and extraction of metadata from json, Agnieszka Mikolajczyk 2019

import urllib.request, json
import sys
import os


# returns the Xeno Canto catalogue numbers for the given list of search terms.
# @param searchTerms: list of search terms
# http://www.xeno-canto.org/explore?query=common+snipe


# Creates the subdirectory data/xeno-canto-dataset if necessary
# Downloads and saves json files for number of pages in a query
# and directory path to saved json's
def save_json(searchTerms, birdName, country):
    numPages = 1
    page = 1
    # create a path to save json files and recordings
    path = "../data/xeno-canto-dataset/" + birdName.replace(':', '') + "/" + country
    if not os.path.exists(path):
        print("Creating subdirectory " + path + " for downloaded files...")
        os.makedirs(path)
        # download a json file for every page found in a query
    while page < numPages + 1:
        print("Loading page " + str(page) + "...")
        url = 'https://www.xeno-canto.org/api/2/recordings?query={0}&page={1}'.format(searchTerms.replace(' ', '%20'),
                                                                                      page)
        print(url)
        jsonPage = urllib.request.urlopen(url)
        jsondata = json.loads(jsonPage.read().decode('utf-8'))
        filename = path + "/jsondata_p" + str(page) + ".json"
        with open(filename, 'w') as outfile:
            json.dump(jsondata, outfile)
        # check number of pages
        numPages = jsondata['numPages']
        page = page + 1
    print("Found ", numPages, " pages in total.")
    # return number of files in json
    # each page contains 500 results, the last page can have less than 500 records
    print("Saved json for ", (numPages - 1) * 500 + len(jsondata['recordings']), " files")
    return path


# reads the json and return the list of values for selected json part
# i.e. "id" - ID number, "type": type of the bird sound such as call or song
# for all Xeno Canto files found with the given search terms.
def read_data(searchTerm, path):
    data = []
    numPages = 1
    page = 1
    # read all pages and save results in a list
    while page < numPages + 1:
        # read file
        with open(path + "/jsondata_p" + str(page) + ".json", 'r') as jsonfile:
            jsondata = jsonfile.read()
        jsondata = json.loads(jsondata)
        # check number of pages
        numPages = jsondata['numPages']
        # find "recordings" in a json and save a list with a search term
        for k in range(len(jsondata['recordings'])):
            data.append(jsondata["recordings"][k][searchTerm])
        page = page + 1
    return data


# downloads all sound files found with the search terms into xeno-canto directory
# into catalogue named after the search term (i.e. Apus apus)
# filename have two parts: the name of the bird in latin and ID number
def download(searchTerms, birdName, country):
    # create data/xeno-canto-dataset directory
    path = save_json(searchTerms, birdName, country)
    # get filenames: recording ID and bird name in latin from json
    filenamesID = read_data('id', path)
    filenamesCountry = read_data('cnt', path)
    # get website recording http download address from json
    fileaddress = read_data('file', path)
    numfiles = len(filenamesID)
    print("A total of ", numfiles, " files will be downloaded")
    for i in range(0, numfiles):
        print("Saving file ", i + 1, "/", numfiles,
              ": data/xeno-canto-dataset/" + birdName.replace(':', '') + filenamesID[
                  i] + ".mp3")
        # Check if fileaddress[i] starts with 'http' or 'https'. If not, prefix the base URL.
        if not fileaddress[i].startswith("http"):
            full_url = "https://www.xeno-canto.org" + fileaddress[i]
        else:
            full_url = fileaddress[i]

        urllib.request.urlretrieve(full_url, path + "/" + birdName + filenamesID[i] + ".mp3")



def main(argv):
    # download all sounds type song, from bird list recorded in selected countries 
    countries = ['Poland', 'Germany', 'Slovakia', 'Czech', 'Lithuania']
    birds = ['Emberiza Citrinella',
             'Parus Major',
             'Phylloscopus Collybita',
             'Sylvia Atricapilla',
             'Acrocephalus Arundinaceus']
    for country in countries:
        for bird in birds:
            download(bird + ' cnt:' + country + ' type:song', bird.replace(' ', ''), country)


if __name__ == "__main__":
    main(sys.argv)

In [ ]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

def extract_mfcc_from_wav(file_path, n_mfcc=13):
    y, sr = librosa.load(file_path, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return np.mean(mfcc.T, axis=0)

def generate_spectrogram(file_path, output_path):
    y, sr = librosa.load(file_path, sr=None)
    plt.figure(figsize=(10, 4))
    D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)
    librosa.display.specshow(D, sr=sr, x_axis='time', y_axis='log')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Spectrogram')
    plt.savefig(output_path)
    plt.close()

def extract_mfcc_from_spectrogram(spectrogram_path, n_mfcc=13):
    # Load the spectrogram as grayscale
    img = plt.imread(spectrogram_path)
    # Convert to grayscale if needed
    if len(img.shape) == 3:
        img = np.mean(img, axis=2)
    return img.flatten()[:n_mfcc]

def prepare_dataset(file_paths, labels):
    features = []
    for file_path in file_paths:
        mfcc_features = extract_mfcc_from_wav(file_path)
        features.append(mfcc_features)
    return np.array(features), np.array(labels)

file_paths = ["bird1.wav", "bird2.wav", "bird3.wav"]
labels = [0, 1, 1] 

# Prepare dataset
X, y = prepare_dataset(file_paths, labels)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Classification using Support Vector Machine (SVM)
classifier = SVC(kernel='rbf')
classifier.fit(X_train, y_train)

# Predictions
y_pred = classifier.predict(X_test)

# Evaluation
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
